In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import matplotlib 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import sys, os

import math
import numpy as np

# Map1D_B
---


In [2]:
from gpt.maps import Map1D_B
from gpt.lattice import Lattice

sol = Map1D_B('sol', 'fields/solenoid_SLA_L60_1D.gdf')
?sol

ImportError: cannot import name 'Lattice' from 'gpt.element' (/Users/colwyngulliford/GitHub/lume-gpt/gpt/element.py)

In [ ]:
sol.plot_floor()

In [ ]:
sol.plot_field_profile()

In [16]:
for line in sol.gpt_lines(): print(line)

sol_x = 0.0;
sol_y = 0.0;
sol_z = 0.3;
sol_scale = 1;
Map1D_B("wcs", sol_x, sol_y, sol_z, 1, 0, 0, 0, 1, 0, "/Users/colwyngulliford/Documents/GitHub/lume-gpt/examples/elements/fields/solenoid_SLA_L60_1D.gdf", "z", "Bz", sol_scale);


In [6]:
?sol.field_integral
sol.field_integral

0.0010917178030000001

Type:        property
String form: <property object at 0x15ff8a9a0>
Docstring:   Returns the on axis integral of the Fz 


In [9]:
?sol.larmor_angle
print('Larmor Angle:', sol.larmor_angle(p=1e6)*180/np.pi,' deg.')

Larmor Angle: -9.376132418274313  deg.


Signature: sol.larmor_angle(p=1000000.0)
Docstring: <no docstring>
File:      ~/Documents/GitHub/lume-gpt/gpt/maps.py
Type:      method
